In [1]:
#pip install neo4j

In [2]:
from neo4j import GraphDatabase
import pandas as pd
from tabulate import tabulate
import os

In [3]:
directory = os.getcwd()
path = directory + '/Data/'

In [4]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

## Connecting to neo4j

To connect to neo4j database, lauch your database in the neo4j application. Use your password in the following statement

driver=GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j","<Password>"))

In [5]:
driver=GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j",""))
session = driver.session(database="graphwooptions")

In [6]:
result={"label":[],"count":[]}
with driver.session(database="graphwooptions") as session:
    labels=[row["label"] for row in session.run("CALL db.labels()")]
    for label in labels:
        query=f"MATCH (:`{label}`) RETURN count(*) as count"
        count=session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
df=pd.DataFrame(data=result)
print(tabulate(df.sort_values("count"), headers='keys',tablefmt='psql',showindex=False))

+--------------------+---------+
| label              |   count |
|--------------------+---------|
| _Neodash_Dashboard |       1 |
| company            |   20974 |
+--------------------+---------+


## Degree centrality: Counts outgoing relationships
Detects the number of direct connections a node has


In [7]:
pd.set_option('display.max_rows', 500)
fig=plt.figure()
query="""CALL gds.degree.stream({
nodeProjection: 'company',
relationshipProjection: {Holds: {type: 'Holds'}}})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS cik,gds.util.asNode(nodeId).company AS name, score as Degree_natural
ORDER BY Degree_natural DESC
"""

with driver.session(database="graphwooptions") as session:
    Degree_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural=Degree_natural[['cik','name','Degree_natural']]
Degree_natural

,cik,name,Degree_natural
0,1000275,ROYAL BK OF CANADA,10846.0
1,1533421,TOWER RESEARCH CAPITAL LLC,8656.0
2,102909,VANGUARD GRP IN,8606.0
3,161052,UBS GRP AG,8575.0
4,70858,BK OF AMERICA CORP DE,7974.0
...,...,...,...
20969,779336,ABERDEEN AUSTRALIA EQUITY FUND INC,0.0
20970,779544,ARK RESTAURANTS CORP,0.0
20971,78003,PFIZER INC,0.0
20972,780571,"ITRON, INC.",0.0


In [8]:
query="""CALL gds.degree.stream({
nodeProjection: 'company',
relationshipProjection: {Holds: {type: 'Holds'}}, orientation: 'REVERSE'  })
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS cik,   score as Degree_reverse
ORDER BY Degree_reverse DESC
"""

with driver.session(database="graphwooptions") as session:
    Degree_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse=Degree_reverse[['cik','Degree_reverse']]
Degree_reverse

,cik,Degree_reverse
0,1100663,58760.0
1,884394,15204.0
2,36405,13531.0
3,1209466,12612.0
4,1454889,9838.0
...,...,...
20969,1728321,0.0
20970,1728354,0.0
20971,1728355,0.0
20972,1728436,0.0


In [9]:
query='''match ()-[r:Holds]->()
where r.value = toString(r.value)
set r.value = toInteger(r.value)
return count(r)'''
with driver.session(database="graphwooptions") as session:
    session.run(query)

In [10]:
query="""CALL gds.degree.stream(({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value'
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Degree_natural_wt
RETURN cik,  Degree_natural_wt
ORDER BY Degree_natural_wt DESC
"""

with driver.session(database="graphwooptions") as session:
    Degree_natural_wt=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_natural_wt=Degree_natural_wt[['cik','Degree_natural_wt']]
Degree_natural_wt["Degree_natural_wt"]=Degree_natural_wt["Degree_natural_wt"].apply(int)

In [11]:
query="""CALL gds.degree.stream(({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value'
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,   score as Degree_reverse_wt
RETURN cik,  Degree_reverse_wt
ORDER BY Degree_reverse_wt DESC
"""

with driver.session(database="graphwooptions") as session:
    Degree_reverse_wt=pd.DataFrame([dict(record) for record in session.run(query)])
    Degree_reverse_wt=Degree_reverse_wt[['cik','Degree_reverse_wt']]
Degree_reverse_wt["Degree_reverse_wt"]=Degree_reverse_wt["Degree_reverse_wt"].apply(int)

## Approx betweeness
calculates shortest paths between a subset of nodes, unlike Betweenness which considers all pairs of nodes

In [12]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  },
  samplingSize: 100
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score 
RETURN cik,  score as Ap_Betweeness_natural
ORDER BY Ap_Betweeness_natural DESC
'''

with driver.session(database="graphwooptions") as session:
    Ap_Betweeness_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Ap_Betweeness_natural=Ap_Betweeness_natural[['cik','Ap_Betweeness_natural']]
Ap_Betweeness_natural

,cik,Ap_Betweeness_natural
0,19617,95879.064441
1,895421,83876.165331
2,1390777,79153.156018
3,1364742,56088.844306
4,161052,49293.952026
...,...,...
20970,1263994,0.000000
20971,1264089,0.000000
20972,1264136,0.000000
20973,1264806,0.000000


In [13]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  },
  samplingSize: 100
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Ap_Betweeness_reverse
ORDER BY Ap_Betweeness_reverse DESC
'''

with driver.session(database="graphwooptions") as session:
    Ap_Betweeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Ap_Betweeness_reverse=Ap_Betweeness_reverse[['cik','Ap_Betweeness_reverse']]
Ap_Betweeness_reverse

,cik,Ap_Betweeness_reverse
0,19617,114971.221654
1,1364742,37932.166798
2,105598,35990.520227
3,895421,35985.188348
4,1390777,32843.285925
...,...,...
20970,1792509,0.000000
20971,751656,0.000000
20972,765878,0.000000
20973,775995,0.000000


## Pagerank 
Measures the transitive influence or connectivity of nodes

In [14]:
query='''CALL gds.pageRank.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  dampingFactor: 0.85,
  maxIterations: 20
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Page_Rank_wt_natural
ORDER BY Page_Rank_wt_natural DESC
'''

with driver.session(database="graphwooptions") as session:
    Page_Rank_wt_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Page_Rank_wt_natural=Page_Rank_wt_natural[['cik','Page_Rank_wt_natural']]
Page_Rank_wt_natural

,cik,Page_Rank_wt_natural
0,1100663,58.640159
1,320193,24.973123
2,884394,23.285977
3,36405,22.822094
4,789019,18.220705
...,...,...
20970,1556915,0.150000
20971,1556921,0.150000
20972,1557017,0.150000
20973,1557406,0.150000


In [15]:
query='''CALL gds.pageRank.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  dampingFactor: 0.85,
  maxIterations: 20
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name,  score
RETURN cik,  score as Page_Rank_wt_reverse
ORDER BY Page_Rank_wt_reverse DESC
'''

with driver.session(database="graphwooptions") as session:
    Page_Rank_wt_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Page_Rank_wt_reverse=Page_Rank_wt_reverse[['cik','Page_Rank_wt_reverse']]
Page_Rank_wt_reverse

,cik,Page_Rank_wt_reverse
0,102909,80.842766
1,1364742,68.417562
2,1318757,51.655390
3,887777,51.419463
4,1214717,42.265412
...,...,...
20970,1784384,0.150000
20971,1788707,0.150000
20972,1794354,0.150000
20973,1796335,0.150000


## HITS
Link analysis algorithm that rates nodes based on two scores, a hub score and an authority score.

In [16]:
query='''CALL gds.alpha.hits.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  hitsIterations: 20
})) YIELD nodeId, values
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, values.auth AS authScore, values.hub AS hubScore
RETURN cik, authScore, hubScore
ORDER BY authScore DESC'''
with driver.session(database="graphwooptions") as session:
    HITS_wt_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    HITS_wt_reverse=HITS_wt_reverse[['cik','authScore','hubScore']]
HITS_wt_reverse

,cik,authScore,hubScore
0,70858,0.247967,0.000001
1,1000275,0.187046,0.004955
2,161052,0.158766,0.003655
3,1677044,0.151697,0.000000
4,895421,0.141981,0.008781
...,...,...,...
20970,1679033,0.000000,0.000825
20971,1679049,0.000000,0.001168
20972,1679082,0.000000,0.001518
20973,1679273,0.000000,0.004229


In [17]:
query='''CALL gds.alpha.hits.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  hitsIterations: 20
})) YIELD nodeId, values
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, values.auth AS authScore, values.hub AS hubScore
RETURN cik, authScore, hubScore
ORDER BY authScore DESC'''
with driver.session(database="graphwooptions") as session:
    HITS_wt_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    HITS_wt_natural=HITS_wt_natural[['cik','authScore','hubScore']]
HITS_wt_natural

,cik,authScore,hubScore
0,1100663,0.843629,0.000000
1,1209466,0.274989,0.000000
2,884394,0.220057,0.000000
3,36405,0.115034,0.000000
4,930667,0.110831,0.000000
...,...,...,...
20970,1480532,0.000000,0.000006
20971,1480751,0.000000,0.000356
20972,1480916,0.000000,0.008757
20973,1481669,0.000000,0.000070


## reverting relationships

In [18]:
# query='''MATCH (c)-[rel:Holds]->(p)
# CALL apoc.refactor.invert(rel)
# yield input, output
# RETURN input, output'''
# with driver.session(database="graphwooptions") as session:
#     session.run(query)

## Closeness, it is unweighted
Detect nodes that are able to spread information very efficiently through a graph

In [19]:
query='''CALL gds.alpha.closeness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  }
})) YIELD nodeId, centrality AS score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, score
RETURN cik,score as Closeness_natural
ORDER BY Closeness_natural DESC
'''
with driver.session(database="graphwooptions") as session:
    Closeness_natural=pd.DataFrame([dict(record) for record in session.run(query)])
    Closeness_natural=Closeness_natural[['cik','Closeness_natural']]
Closeness_natural

,cik,Closeness_natural
0,1611326,1.000000
1,1611737,1.000000
2,19617,0.585079
3,895421,0.560302
4,1364742,0.557843
...,...,...
20970,743241,0.229553
20971,1756161,0.228477
20972,1114643,0.225711
20973,1732988,0.207551


In [20]:
query='''CALL gds.alpha.closeness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  }
})) YIELD nodeId, centrality AS score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, score
RETURN cik, score as Closeness_reverse
ORDER BY Closeness_reverse DESC
'''
with driver.session(database="graphwooptions") as session:
    Closeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Closeness_reverse=Closeness_reverse[['cik','Closeness_reverse']]
Closeness_reverse

,cik,Closeness_reverse
0,1611326,1.000000
1,1611737,1.000000
2,19617,0.585079
3,895421,0.560302
4,1364742,0.557843
...,...,...
20970,743241,0.229553
20971,1756161,0.228477
20972,1114643,0.225711
20973,1732988,0.207551


In [21]:
query='''CALL gds.alpha.closeness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'UNDIRECTED',
      properties: {}
    }
  }
})) YIELD nodeId, centrality AS score
WITH gds.util.asNode(nodeId).id AS cik, gds.util.asNode(nodeId).company AS name, score
RETURN cik, score as Closeness_und
ORDER BY Closeness_und DESC
'''
with driver.session(database="graphwooptions") as session:
    Closeness_und=pd.DataFrame([dict(record) for record in session.run(query)])
    Closeness_und=Closeness_und[['cik','Closeness_und']]
Closeness_und

,cik,Closeness_und
0,1611326,1.000000
1,1611737,1.000000
2,19617,0.585079
3,895421,0.560302
4,1364742,0.557843
...,...,...
20970,743241,0.229553
20971,1756161,0.228477
20972,1114643,0.225711
20973,1732988,0.207551


## Betweenness(unweighted)
A way of detecting the amount of influence a node has over the flow of information in a graph


In [22]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'NATURAL',
      properties: {}
    }
  }
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Betweeness
RETURN cik,Betweeness
ORDER BY Betweeness DESC
'''

with driver.session(database="graphwooptions") as session:
    Betweeness=pd.DataFrame([dict(record) for record in session.run(query)])
    Betweeness=Betweeness[['cik','Betweeness']]
Betweeness["Betweeness"]=Betweeness["Betweeness"].apply(int)

In [23]:
query='''CALL gds.betweenness.stream(({
  nodeProjection: '*',
  relationshipProjection: {
    relType: {
      type: '*',
      orientation: 'REVERSE',
      properties: {}
    }
  }
})) YIELD nodeId, score
WITH gds.util.asNode(nodeId).id AS cik,  score as Betweeness_reverse
RETURN cik, Betweeness_reverse
ORDER BY Betweeness_reverse DESC
'''

with driver.session(database="graphwooptions") as session:
    Betweeness_reverse=pd.DataFrame([dict(record) for record in session.run(query)])
    Betweeness_reverse=Betweeness_reverse[['cik','Betweeness_reverse']]
Betweeness_reverse["Betweeness_reverse"]=Betweeness_reverse["Betweeness_reverse"].apply(int)

## Joining all the tables

In [24]:
dfs = [Degree_natural, Degree_reverse, Degree_natural_wt, Degree_reverse_wt, Betweeness, Betweeness_reverse, Ap_Betweeness_natural, Ap_Betweeness_reverse, Page_Rank_wt_natural, Page_Rank_wt_reverse, HITS_wt_reverse, HITS_wt_natural, Closeness_natural, Closeness_reverse,Closeness_und]

In [25]:
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cik'), dfs)

In [27]:
df_final.to_csv(path + 'df_final_graph_wo_options.csv', index=False)